# Baseline code for DataDrive2030 Early Learning Predictors Baseline CHALLENGE AT ZIndi: https://zindi.africa/competitions/datadrive2030-early-learning-predictors-challenge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.2 MB/s eta 0:00:00


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Train = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/Train.csv')
Test = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/Test_zindi_nn.csv')
SampleSubmission = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/SampleSubmission.csv')
VariableDescription = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/VariableDescription.csv')

<ipython-input-19-983632fd1800>:1: DtypeWarning: Columns (140,172,254,257,275,306,343,358,373,374,461,635,641,642,643) have mixed types. Specify dtype option on import or set low_memory=False.
  Train = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/Train.csv')
<ipython-input-19-983632fd1800>:2: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types. Specify dtype option on import or set low_memory=False.
  Test = pd.read_csv('/content/drive/MyDrive/zindi/zindi_DataDrive2030 Early Learning Predictors Challenge/Test_zindi_nn.csv')


In [ ]:
Train.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
0,ID_SYSJ2FM0D,2022.0,2022-02-03,59.000000,NaN,NaN,NaN,NaN,NaN,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.500000
1,ID_J5BTFOZR3,2019.0,NaN,60.163933,NaN,NaN,NaN,1st year in the programme,103.000000,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.869999
2,ID_R00SN7AUD,2022.0,2022-03-11,69.000000,NaN,NaN,NaN,NaN,108.400002,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.520000
3,ID_BSSK60PAZ,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.599998
4,ID_IZTY6TC4D,2021.0,2021-10-13,57.000000,2021-10-13,0.0,NaN,2nd year in programme,114.000000,Almost always,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.599998


In [ ]:
Train.isnull().sum(), Train.shape

(child_id                   0
 data_year                  0
 child_date              1821
 child_age                  0
 child_enrolment_date    5964
                         ... 
 obs_heating_4           6324
 obs_heating_5           6324
 obs_heating_6           6324
 obs_heating_7           6324
 target                     0
 Length: 679, dtype: int64,
 (8585, 679))

In [ ]:
Test.isnull().sum(), Test.shape

(child_id                   0
 data_year                  0
 child_date               754
 child_age                  0
 child_enrolment_date    2568
                         ... 
 obs_heating_3           2712
 obs_heating_4           2712
 obs_heating_5           2712
 obs_heating_6           2712
 obs_heating_7           2712
 Length: 678, dtype: int64,
 (3680, 678))

In [ ]:
SampleSubmission

In [ ]:
VariableDescription

In [ ]:
not_features

['child_attends',
 'child_attendance',
 'child_languages',
 'pqa_date',
 'pqa_class_age',
 'pqa_class_age_1',
 'pqa_class_age_2',
 'pqa_class_age_3',
 'pqa_class_age_4',
 'pqa_class_age_5',
 'pqa_class_age_6',
 'pqa_class_assistants',
 'pqa_environment_areas',
 'pqa_environment_variety',
 'pqa_environment_appropriate',
 'pqa_environment_accessible',
 'pqa_environment_open',
 'pqa_environment_outdoor',
 'pqa_score_environment',
 'pqa_assessment_observation',
 'pqa_assessment_systematic',
 'pqa_score_assessment',
 'pqa_relationships_peers',
 'pqa_relationships_staff',
 'pqa_relationships_acknowledge',
 'pqa_relationships_discipline',
 'pqa_score_relationships',
 'pqa_curriculum_ncf',
 'pqa_curriculum_plan',
 'pqa_curriculum_balance',
 'pqa_curriculum_numeracy',
 'pqa_curriculum_literacy',
 'pqa_score_curriculum',
 'pqa_teaching_choice',
 'pqa_teaching_engagement',
 'pqa_teaching_participation',
 'pqa_teaching_questions',
 'pqa_teaching_support',
 'pqa_score_teaching',
 'pqa_class',
 'pra

In [ ]:
#pri_toys
features = []; cat_features = []; not_features = []
for k in Train.columns[1:]:
    if Train[k].isnull().sum() < 6000:
        features.append(k)
        if Train[k].dtype == 'O':
            cat_features.append(k)
            print('There is '+ str(len(Train[k].value_counts()))+' Class in: ' +k)
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' features that have more than 6000 of missing values')

There is 254 Class in: child_date
There is 535 Class in: child_enrolment_date
There is 4 Class in: child_grant
There is 4 Class in: child_years_in_programme
There is 4 Class in: child_observe_attentive
There is 4 Class in: child_observe_concentrated
There is 4 Class in: child_observe_diligent
There is 4 Class in: child_observe_interested
There is 2 Class in: child_gender
There is 1018 Class in: child_dob
There is 3 Class in: child_stunted
There is 4 Class in: child_age_group
There is 153 Class in: id_mn_best
There is 10 Class in: prov_best
There is 50 Class in: id_dc_best
There is 50 Class in: dc_best
There is 153 Class in: mn_best
There is 5 Class in: pra_free_play
There is 4 Class in: pra_free_play_outdoor
There is 31 Class in: pra_groupings
There is 2 Class in: pra_groupings_1
There is 2 Class in: pra_groupings_2
There is 2 Class in: pra_groupings_3
There is 2 Class in: pra_groupings_4
There is 2 Class in: pra_groupings_5
There is 3 Class in: pra_engaged
There is 4 Class in: pra_age

In [ ]:
Train = Train[features]
Test  = Test[features[:-1]]

In [ ]:
max(Train['target']), min(Train['target'])

(96.80999755859376, 6.369999885559082)

In [ ]:
Train.shape

(8585, 277)

In [ ]:
Train = Train.fillna('')
Test  = Test.fillna('')

In [ ]:
train_new =

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
for fold ,(train_indicies,valid_indicies) in enumerate(kfold.split(X=Train, y = Train['target'])):
  Train.loc[valid_indicies,'kfold']=fold

<ipython-input-25-4d4fbf19992a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train.loc[valid_indicies,'kfold']=fold


In [ ]:
Train.head()

,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,...,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,ses_cat,target,kfold,id
0,2022.0,2022-02-03,59.000000,,,,,,Sometimes,Sometimes,...,Yes,7.0,14.0,107.0,134.0,1051.0,,51.500000,3.0,0
1,2019.0,,60.163933,,,,1st year in the programme,103.0,Sometimes,Almost never,...,Yes,24.0,,,367.0,1832.0,,55.869999,2.0,1
2,2022.0,2022-03-11,69.000000,,,,,108.400002,Often,Often,...,No,8.0,24.0,1448.0,1448.0,3214.0,R0-110,47.520000,4.0,2
3,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,Almost always,...,No,4.0,22.0,76.0,629.0,3214.0,R291-750,58.599998,2.0,3
4,2021.0,2021-10-13,57.000000,2021-10-13,0.0,,2nd year in programme,114.0,Almost always,Almost always,...,No,1.0,30.0,71.0,315.0,1832.0,R0-110,76.599998,1.0,4


In [ ]:
Train['id'] = Train.index

<ipython-input-14-2b6b806c6dae>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['id'] = Train.index


In [ ]:
ff=['id','target','kfold','pri_toys','count_staff_gender_other', 'language_match']
useful_feature=[c for c in Train.columns if c not in ff]

In [ ]:
len(cat_features)

198

In [ ]:
Train.shape

(8585, 278)

In [ ]:
cat_features.remove('pri_toys')
cat_features.remove('count_staff_gender_other')
cat_features.remove('language_match')

In [ ]:
#pri_toys
features = []; cat_features = []; not_features = []
for k in Train[useful_feature].columns[1:]:
    if Train[k].isnull().sum() < 6000:
        features.append(k)
        if Train[k].dtype == 'O':
            cat_features.append(k)
            print('There is '+ str(len(Train[k].value_counts()))+' Class in: ' +k)
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' features that have more than 6000 of missing values')

There is 255 Class in: child_date
There is 536 Class in: child_enrolment_date
There is 59 Class in: child_months_enrolment
There is 5 Class in: child_grant
There is 5 Class in: child_years_in_programme
There is 583 Class in: child_height
There is 4 Class in: child_observe_attentive
There is 4 Class in: child_observe_concentrated
There is 4 Class in: child_observe_diligent
There is 4 Class in: child_observe_interested
There is 14 Class in: child_observe_total
There is 2 Class in: child_gender
There is 1019 Class in: child_dob
There is 4510 Class in: child_zha
There is 4 Class in: child_stunted
There is 4 Class in: child_age_group
There is 154 Class in: id_mn_best
There is 10 Class in: prov_best
There is 51 Class in: id_dc_best
There is 51 Class in: dc_best
There is 154 Class in: mn_best
There is 98 Class in: ward_best
There is 213 Class in: id_enumerator
There is 2083 Class in: id_facility
There is 6 Class in: pra_free_play
There is 5 Class in: pra_free_play_outdoor
There is 32 Class in

In [ ]:
Train.shape

(8585, 278)

In [ ]:
preds_test

array([61.45892181, 46.17579319, 47.52273382, ..., 38.41453128,
       35.88288449, 39.06216014])

In [ ]:
useful_feature

In [ ]:
final_test_pr = []
final_test_pr.append(preds_test)

In [ ]:
xtrain['']

,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,...,gps_ind,pre_covid,ses_proxy,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,ses_cat
0,2022.0,2022-02-03,59.000000,,,,,,Sometimes,Sometimes,...,Yes,Post COVID,2.0,Yes,7.0,14.0,107.0,134.0,1051.0,
1,2019.0,,60.163933,,,,1st year in the programme,103.0,Sometimes,Almost never,...,No,Pre-COVID,4.0,Yes,24.0,,,367.0,1832.0,
2,2022.0,2022-03-11,69.000000,,,,,108.400002,Often,Often,...,Yes,Post COVID,1.0,No,8.0,24.0,1448.0,1448.0,3214.0,R0-110
3,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,Almost always,...,Yes,Post COVID,3.0,No,4.0,22.0,76.0,629.0,3214.0,R291-750
4,2021.0,2021-10-13,57.000000,2021-10-13,0.0,,2nd year in programme,114.0,Almost always,Almost always,...,Yes,Post COVID,1.0,No,1.0,30.0,71.0,315.0,1832.0,R0-110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6863,2022.0,2022-03-11,64.000000,,,No,,109.400002,Almost always,Almost always,...,Yes,Post COVID,1.0,No,8.0,24.0,1448.0,1448.0,3214.0,R0-110
6864,2021.0,2021-11-10,55.000000,2021-01-12,9.0,Yes,1st year in the programme,102.300003,Often,Often,...,Yes,Post COVID,2.0,No,9.0,21.0,65.0,139.0,777.0,R111-290
6865,2021.0,2021-10-12,55.000000,2019-01-15,32.0,,2nd year in programme,102.599998,Often,Sometimes,...,Yes,Post COVID,2.0,No,4.0,8.0,58.0,695.0,1408.0,R111-290
6866,2021.0,2021-11-02,56.000000,2018-01-07,45.0,,3rd year in programme,103.800003,Almost always,Almost always,...,Yes,Post COVID,4.0,No,4.0,14.0,73.0,119.0,868.0,R751-1750


In [ ]:
cat_features = cat_features.remove('pri_toys')

In [ ]:
 cat_features

['child_date',
 'child_enrolment_date',
 'child_grant',
 'child_years_in_programme',
 'child_observe_attentive',
 'child_observe_concentrated',
 'child_observe_diligent',
 'child_observe_interested',
 'child_gender',
 'child_dob',
 'child_stunted',
 'child_age_group',
 'id_mn_best',
 'prov_best',
 'id_dc_best',
 'dc_best',
 'mn_best',
 'pra_free_play',
 'pra_free_play_outdoor',
 'pra_groupings',
 'pra_groupings_1',
 'pra_groupings_2',
 'pra_groupings_3',
 'pra_groupings_4',
 'pra_groupings_5',
 'pra_engaged',
 'pra_agency_choice',
 'pra_agency_explore',
 'pra_agency_questions',
 'pra_agency_understand',
 'pra_agency_play',
 'pra_agency_learn',
 'pra_agency_order',
 'pra_plans',
 'pra_plans_1',
 'pra_plans_2',
 'pra_plans_3',
 'pra_plans_0',
 'pra_ind',
 'pri_mobile',
 'pri_school',
 'pri_holidays',
 'pri_calc_time_open',
 'pri_calc_time_close',
 'pri_separate',
 'pri_language',
 'pri_language_1',
 'pri_language_2',
 'pri_language_3',
 'pri_language_4',
 'pri_language_5',
 'pri_language

In [ ]:
features = []; cat_features = []; not_features = []
for k in Train.columns[1:]:
    if Train[k].isnull().sum() < 6000:
        features.append(k)
        if Train[k].dtype == 'O':
            cat_features.append(k)
            print('There is '+ str(len(Train[k].value_counts()))+' Class in: ' +k)
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' features that have more than 6000 of missing values')

In [ ]:
Train['id'] = Train.index

<ipython-input-33-2b6b806c6dae>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['id'] = Train.index


In [ ]:
cat_features_new = list(set(cat_features).intersection(uuu))

In [ ]:
uuu

array(['child_observe_diligent', 'child_observe_attentive', 'prov_best',
       'child_age', 'child_observe_concentrated',
       'teacher_emotional_total', 'id_enumerator',
       'child_years_in_programme', 'obs_materials', 'child_height',
       'pri_fees_amount_pv', 'child_zha', 'obs_handwashing_1',
       'child_observe_interested', 'child_age_group', 'child_date',
       'count_register_race_other', 'obs_toilet_6', 'pri_year',
       'pri_capacity', 'child_observe_total', 'teacher_duration',
       'count_register_race_white', 'id_ward', 'id_facility_n',
       'teacher_emotional_understand', 'child_gender', 'id_facility',
       'count_toilets_children', 'teacher_social_assistance',
       'child_stunted', 'id_dc_best', 'id_team',
       'certificate_registration_program', 'ward_best', 'pri_attendance',
       'pri_fees_amount', 'child_months_enrolment', 'ses_cat',
       'pri_calc_time_open', 'longitude', 'latitude', 'id_mn_best',
       'language_assessment', 'count_staff_qual

In [ ]:
final_test_pr = []
final_valid_pr={}
df=Train.copy()
scores=[]
tt=Test.copy()
useful_feature=uuu.copy()
from sklearn.model_selection import train_test_split
scores=[]
final_valid_pr={}
for fold in range(5):
  xtrain=df[df.kfold!=fold].reset_index(drop=True)  #divding the dataset selecting subsample for training
  xvalid=df[df.kfold==fold].reset_index(drop=True)

  valid_ids=xvalid.id.values.tolist()


  ytrain=xtrain['target']
  yvalid=xvalid['target']

  xtrain=xtrain[useful_feature]
  xvalid=xvalid[useful_feature]
                        #by turbine ,saving their id for ordering them for final
                              #submission
  #xtrain, xvalid, ytrain, yvalid = train_test_split(
    #xtrain_new[useful_feature],
    #xtrain_new['Target'],
    #test_size    = 0.1,                                          #validation part
    #random_state = 42,
    #shuffle      = True)

  lgbm = CatBoostRegressor(iterations = 4000, learning_rate=0.1, random_seed=123, cat_features=cat_features_new)
  lgbm.fit(xtrain, ytrain ,eval_set=[(xvalid, yvalid)],verbose=20, use_best_model=True, early_stopping_rounds=500)

  preds_valid = lgbm.predict(xvalid)

  rmse=np.sqrt(mean_squared_error(yvalid,preds_valid))
  print(fold,rmse)
  scores.append(rmse)
  preds_test = lgbm.predict(tt[useful_feature])
  final_test_pr.append(preds_test)
  #final_test_value.append(test_p)


  final_valid_pr.update(dict(zip(valid_ids,preds_valid)))

  #scores.append(f1)

print(np.mean(scores),np.std(scores))

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['ID','tt_pr']
final_valid_pred.to_csv('cat_1.csv',index=False)


0:	learn: 14.7246680	test: 14.6097872	best: 14.6097872 (0)	total: 291ms	remaining: 19m 22s
20:	learn: 11.4883588	test: 11.6644421	best: 11.6644421 (20)	total: 3.11s	remaining: 9m 49s
40:	learn: 10.8469391	test: 11.2208466	best: 11.2208466 (40)	total: 4.41s	remaining: 7m 5s
60:	learn: 10.5362604	test: 11.0597359	best: 11.0597359 (60)	total: 5.75s	remaining: 6m 11s
80:	learn: 10.2513034	test: 10.8902333	best: 10.8902333 (80)	total: 7.06s	remaining: 5m 41s
100:	learn: 10.0445056	test: 10.7701736	best: 10.7701736 (100)	total: 8.43s	remaining: 5m 25s
120:	learn: 9.8341733	test: 10.6465938	best: 10.6465938 (120)	total: 9.78s	remaining: 5m 13s
140:	learn: 9.6464104	test: 10.5602656	best: 10.5602656 (140)	total: 11.1s	remaining: 5m 4s
160:	learn: 9.5384982	test: 10.5193567	best: 10.5193567 (160)	total: 12.4s	remaining: 4m 55s
180:	learn: 9.3736295	test: 10.4448357	best: 10.4448357 (180)	total: 14.3s	remaining: 5m 1s
200:	learn: 9.2433501	test: 10.3872183	best: 10.3872183 (200)	total: 16.9s	rem

In [ ]:
final_test_pr

[array([59.26116162, 46.62369388, 45.91663802, ..., 41.39283095,
        35.92062809, 44.91418663]),
 array([61.74620912, 49.19717786, 48.08076258, ..., 45.22690727,
        33.75797586, 42.11387107]),
 array([58.06355623, 44.08690184, 49.75124442, ..., 47.92764706,
        38.15859817, 39.1970253 ]),
 array([58.75942365, 46.31958408, 48.96370645, ..., 46.61811616,
        37.4992835 , 44.35186243]),
 array([59.58684178, 47.96060218, 47.47499287, ..., 44.31024916,
        34.46575241, 41.12998914])]

In [ ]:
SampleSubmission['cat_p_f']=np.mean(np.column_stack(final_test_pr),axis=1)

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    Train[Train.columns[:-1]],
    Train[Train.columns[-1]],
    test_size = 0.15,
    random_state = 42,
    shuffle = True
)

In [ ]:
type(preds_test)

numpy.ndarray

In [ ]:
xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape

NameError: ignored

In [ ]:
xtest = Test[features[:-1]]
train_dataset = Pool(data = xtrain, label = ytrain, cat_features=cat_features)
val_dataset   = Pool(data = xvalid, label = yvalid, cat_features=cat_features)
model         = CatBoostRegressor(iterations = 30000, learning_rate=0.1, random_seed=123, verbose=300)
model.fit(train_dataset, eval_set=val_dataset, use_best_model=True, early_stopping_rounds=300)
preds_valid = model.predict(xvalid)
preds_test  = model.predict(xtest)
print(np.sqrt(mean_squared_error(yvalid, preds_valid)))

In [ ]:
pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 17.4 MB/s eta 0:00:00


In [ ]:
import shap
expaliner = shap.TreeExplainer(lgbm)
shap_values = expaliner.shap_values(tt[useful_feature])

In [ ]:
features = tt[useful_feature].columns

In [ ]:
f = {
    'f1' : [], 'f2': [], 'f3': [], 'f4': [],'f5': [],
    'f6' : [], 'f7': [], 'f8': [], 'f9': [],'f10': [],
    'f11' : [], 'f12': [], 'f13': [], 'f14': [],'f15': []
}

In [ ]:
for shap_value in shap_values:
    arr = np.argsort(shap_value)[::-1][:15]
    for ind, a in enumerate(arr):
        name_f = f'f{ind+1}'
        f[name_f].append(features[a])

In [ ]:
SampleSubmission['target']    = np.mean(np.column_stack(final_test_pr),axis=1)
SampleSubmission

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,cat_p_f
0,ID_0I0999N6S,59.483438,child_observe_diligent,child_observe_attentive,teacher_emotional_total,child_observe_total,child_observe_interested,child_years_in_programme,count_toilets_children,child_gender,id_mn_best,pra_ind,ses_cat,pri_fees_amount,teacher_social_peers,teacher_emotional_selfstarter,mn_best,59.483438
1,ID_GQ6ONJ4FP,46.837592,child_observe_attentive,teacher_emotional_total,child_observe_diligent,child_observe_total,child_observe_concentrated,child_observe_interested,pri_fees_amount,teacher_emotional_understand,teacher_social_peers,teacher_emotional_independent,obs_toilet_1,teacher_emotional_selfstarter,child_gender,teacher_social_met,obs_equipment_2,46.837592
2,ID_YZ76CVRW3,48.037469,prov_best,child_observe_diligent,child_date,id_enumerator,id_facility_n,obs_handwashing_1,id_dc_best,child_observe_attentive,id_mn_best,obs_building,id_team,mn_best,obs_materials_5,id_prov,count_register_year_2017,48.037469
3,ID_BNINCRXH8,69.899649,child_observe_diligent,child_observe_attentive,child_observe_concentrated,id_enumerator,child_age,child_observe_total,prov_best,child_date,child_years_in_programme,language_child,teacher_social_assistance,mn_best,pri_capacity,id_mn_best,id_team,69.899649
4,ID_1U7GDTLRI,44.325771,child_observe_diligent,child_observe_total,prov_best,pri_attendance,pri_registered_programme,child_observe_interested,id_dc_best,id_ward,id_mn_best,id_team,id_prov,count_register_year_2020,id_facility,child_date,teacher_emotional_understand,44.325771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,49.137335,child_age,child_age_group,child_observe_attentive,child_observe_interested,child_date,child_stunted,prov_best,id_team,id_ward,child_years_in_programme,id_dc_best,teacher_social_peers,ses_proxy,mn_best,child_observe_concentrated,49.137335
3676,ID_H2RKKMMKK,50.255566,child_age,child_age_group,ward_best,id_enumerator,child_height,obs_handwashing_1,id_ward,child_zha,prov_best,child_gender,id_mn_best,id_team,id_dc_n,count_register_year_2020,pri_meal_2,50.255566
3677,ID_VY8KX7YTZ,45.095150,child_observe_attentive,child_observe_total,child_observe_concentrated,child_observe_diligent,child_date,child_age,child_gender,pri_holidays,pra_ind,child_observe_interested,urban,certificate_registration_program,pra_plans_2,count_register_year_2020,obs_area,45.095150
3678,ID_EO2MYZ4M7,35.960448,child_observe_interested,id_dc_best,latitude,obs_handwashing_1,pri_language_7,ward_best,id_mn_best,count_register_year_2020,pri_meal_2,longitude,child_gender,id_enumerator,certificate_registration_program,teacher_emotional_understand,teacher_social_assistance,35.960448


In [ ]:
len(f['f1'])

3680

In [ ]:
SampleSubmission['target']    = np.mean(np.column_stack(final_test_pr),axis=1)
SampleSubmission['feature_1'] = f['f1']
SampleSubmission['feature_2'] = f['f2']
SampleSubmission['feature_3'] = f['f3']
SampleSubmission['feature_4'] = f['f4']
SampleSubmission['feature_5'] = f['f5']
SampleSubmission['feature_6'] = f['f6']
SampleSubmission['feature_7'] = f['f7']
SampleSubmission['feature_8'] = f['f8']
SampleSubmission['feature_9'] = f['f9']
SampleSubmission['feature_10'] = f['f10']
SampleSubmission['feature_11'] = f['f11']
SampleSubmission['feature_12'] = f['f12']
SampleSubmission['feature_13'] = f['f13']
SampleSubmission['feature_14'] = f['f14']
SampleSubmission['feature_15'] = f['f15']



In [ ]:
uuu = SampleSubmission['feature_1'].unique()

In [ ]:
uuu

array(['child_observe_diligent', 'child_observe_attentive', 'prov_best',
       'child_age', 'child_observe_concentrated',
       'teacher_emotional_total', 'id_enumerator',
       'child_years_in_programme', 'obs_materials', 'child_height',
       'pri_fees_amount_pv', 'child_zha', 'obs_handwashing_1',
       'child_observe_interested', 'child_age_group', 'child_date',
       'count_register_race_other', 'obs_toilet_6', 'pri_year',
       'pri_capacity', 'child_observe_total', 'teacher_duration',
       'count_register_race_white', 'id_ward', 'id_facility_n',
       'teacher_emotional_understand', 'child_gender', 'id_facility',
       'count_toilets_children', 'teacher_social_assistance',
       'child_stunted', 'id_dc_best', 'id_team',
       'certificate_registration_program', 'ward_best', 'pri_attendance',
       'pri_fees_amount', 'child_months_enrolment', 'ses_cat',
       'pri_calc_time_open', 'longitude', 'latitude', 'id_mn_best',
       'language_assessment', 'count_staff_qual

In [ ]:
SampleSubmission

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,cat_p_f
0,ID_0I0999N6S,59.483438,teacher_emotional_total,child_observe_diligent,child_observe_attentive,child_observe_interested,child_observe_total,obs_materials,count_toilets_children,child_years_in_programme,id_mn_best,ses_cat,id_facility,child_gender,id_dc_best,teacher_social_peers,count_children_present,59.483438
1,ID_GQ6ONJ4FP,46.837592,teacher_emotional_total,child_observe_attentive,child_observe_diligent,child_observe_total,child_observe_concentrated,child_observe_interested,teacher_social_total,pri_attendance,count_children_present,teacher_social_peers,certificate_registration_program,child_gender,id_facility_n,pri_fees_amount_pv,pri_calc_time_open,46.837592
2,ID_YZ76CVRW3,48.037469,child_observe_diligent,id_enumerator,prov_best,child_date,child_observe_attentive,id_dc_best,teacher_social_peers,pri_fees_amount_pv,latitude,id_mn_best,id_team,obs_handwashing_1,teacher_emotional_understand,obs_materials_5,child_observe_total,48.037469
3,ID_BNINCRXH8,69.899649,child_observe_diligent,id_enumerator,teacher_duration,child_observe_attentive,child_observe_concentrated,teacher_emotional_total,child_years_in_programme,child_observe_total,child_age,prov_best,child_date,id_facility_n,id_mn_best,language_child,child_zha,69.899649
4,ID_1U7GDTLRI,44.325771,child_observe_diligent,prov_best,pri_registered_programme,child_observe_interested,id_dc_best,pri_attendance,child_observe_attentive,longitude,id_facility,id_ward,child_observe_total,latitude,teacher_emotional_understand,pri_year,id_mn_best,44.325771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,49.137335,child_age,child_age_group,child_years_in_programme,child_date,child_observe_attentive,language_assessment,child_observe_interested,child_zha,prov_best,id_mn_best,id_facility_n,teacher_social_peers,count_register_all,obs_toilet_6,teacher_social_assistance,49.137335
3676,ID_H2RKKMMKK,50.255566,child_age,child_age_group,child_height,id_enumerator,obs_handwashing_1,id_mn_best,ward_best,prov_best,child_zha,id_dc_best,longitude,child_stunted,child_gender,count_children_present,id_ward,50.255566
3677,ID_VY8KX7YTZ,45.095150,child_observe_diligent,child_observe_attentive,child_observe_total,child_observe_interested,child_age,child_observe_concentrated,count_register_all,child_gender,teacher_social_peers,pri_calc_time_open,pri_attendance,child_date,pri_meal_2,teacher_emotional_understand,child_enrolment_date,45.095150
3678,ID_EO2MYZ4M7,35.960448,id_enumerator,pri_meal_2,obs_handwashing_1,longitude,child_zha,id_mn_best,language_child,pri_capacity,id_facility,pri_fees_amount,count_children_present,certificate_registration_program,child_observe_interested,pri_calc_time_open,id_team,35.960448


In [ ]:
SampleSubmission.to_csv('submission_9.87.csv', index=False)